**Deskripsi Kolom**

auction_id: id unik user online yang telah disajikan kuesioner biodata.
 
experiment: kelompok user - control atau exposed.
* control: user yang diperlihatkan iklan biasa.
* exposed: user yang diperlihatkan iklan kreatif atau iklan interaktif online, dengan merek SmartAd.

date: tanggal dengan format YYYY-MM-DD.
 
hour: jam dalam format HH.

device_make: nama jenis perangkat yang digunakan user

platform_os: id OS yang dimiliki user

browser: nama browser yang digunakan user untuk melihat kuesioner biodata.

yes: 1 jika user memilih tombol "Yes" untuk kuesioner biodata.

no: 1 jika user memilih tombol "No" untuk kuesioner biodata.

(kolom baru) success : 1 jika user memilih tombol "Yes", 0 jika user memilih tombol "No"

# Problem

Perusahaan menginginkan sebuah iklan efektif yang mampu menarik banyak user untuk mengisi kuesioner biodata. Perusahaan dihadapkan pada dua pilihan yaitu menggunakan iklan biasa (seperti gambar saja) atau menggunakan iklan interaktif online dengan merek SmartAd. Perusahaan telah melakukan eksperimen dan mengumpulkan data dengan membagi menjadi 2 kelompok user :

* Control : user yang diperlihatkan iklan biasa
* Exposed : user yang diperlihatkan iklan interaktif online

Perusahaan ingin mengetahui apakah iklan interaktif online memiliki tingkat kesuksesan yang lebih tinggi daripada iklan biasa. Tingkat kesuksesan yang dimaksud adalah rasio banyaknya user yang memilih tombol "Yes" untuk mengisi kuesioner biodata terhadap total user yang memilih tombol.

# Import Libraries & Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency

In [ ]:
ad = pd.read_csv('../input/ad-ab-testing/AdSmartABdata - AdSmartABdata.csv')
ad.head()

# Data Pre-processing & EDA

In [ ]:
ad.shape

In [ ]:
ad.isnull().values.any()

Tidak ada missing values pada data sehingga terdapat 8077 user yang mengikuti testing ini. 

In [ ]:
ad[(ad['yes'] == 1) & (ad['no'] == 1)].shape[0]

Tidak ada anomali pada data misalnya seperti user yang memiliki nilai 1 pada kolom 'Yes' dan 'No' karena tidak mungkin terjadi.

In [ ]:
no_response = round(100 * ad[(ad['yes'] == 0) & (ad['no'] == 0)].shape[0] / ad.shape[0], 2)
print(f'{no_response}%')

Sangat banyak sekali user yang tidak merespon hingga 84.61%. User dapat melihat kuesioner biodata dan memungkinkan untuk tidak menanggapi sehingga kolom Yes' dan 'No' nilainya 0. Namun karena testing ini ingin melihat apakah terdapat perbedaan yang signifikan pada tingkat kesuksesan rasio user yang memilih tombol "Yes" terhadap total user yang memilih tombol, maka sebanyak 84.61% baris akan dihapus. 

In [ ]:
ad.drop(ad[(ad['yes'] == 0) & (ad['no'] == 0)].index, inplace = True)

Total user sekarang jumlahnya 1243.

In [ ]:
ad.duplicated('auction_id').sum()

Tidak ada duplikat pada kolom 'auction_id'.

In [ ]:
#Mengubah kolom 'Yes' menjadi success: 1 jika user memilih tombol "Yes", 0 jika user memilih tombol "No"
ad.drop('no', axis = 1, inplace = True)
ad.rename(columns = {'yes' : 'success'}, inplace = True)
ad.head()

In [ ]:
plt.figure(figsize = (10, 5))
ax = sns.countplot(x = 'experiment', data = ad, hue = 'success')
for p in ax.patches:
   ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+0.02))

Perbedaan jumlah user pada kelompok exposed dan control tidak banyak sekali.

In [ ]:
for i in ad['experiment'].unique():
    ad[ad['experiment'] == i]['success'].value_counts().plot.pie(autopct = '%1.f%%')
    plt.title(str(i) + ' group')
    plt.show()

Berdasarkan visualisasi diatas, terdapat sedikit perbedaan antara success rate pada kelompok exposed dengan kelompok control. Selanjutnya akan dilakukan uji hipotesis untuk melihat apakah terdapat perbedaan yang signifikan secara statistik.

# Hypothesis Testing

Gunakan pearson's chi square test karena :
* metrik diskrit (0/1).
* jumlah sampel yang besar (>30).

Hipotesis (two-tailed test)
* H0: there is no significant difference between the success rate of advertising in the control and exposed group.
* H1: there is significant difference between the success rate of advertising in the control and exposed group.

Pengambilan keputusan berdasarkan alpha (0.05):
* Jika p-value <= alpha, maka tolak H0.
* Jika p-value > alpha, maka terima H0.

In [ ]:
#Membuat tabel exposed dan control group terpisah
exposed = ad[ad['experiment'] == 'exposed']
control = ad[ad['experiment'] == 'control']

In [ ]:
#Jumlah user berdasarkan kelompok eksperimen
obs_exposed = exposed['success'].count()
obs_control = control['success'].count()

#Jumlah user yang memilih tombol "Yes" berdasarkan kelompok eksperimen
success_exposed = exposed['success'].sum()
success_control = control['success'].sum()

#Jumlah user yang memilih tombol "No" berdasarkan kelompok eksperimen
fail_exposed = obs_exposed - success_exposed
fail_control = obs_control - success_control

In [ ]:
obs = np.array([[fail_control, fail_exposed],
                [success_control, success_exposed]])
obs

In [ ]:
#Pearson's Chi Square Test
chi2, p, dof, expected = stats.chi2_contingency(obs)

alpha = 0.05

print(f"""chi2 : {chi2}

p-value : {p}
{'reject H0' if p <= alpha else 'accept H0'}

df : {dof}

expected : 
{expected}""")

# Conclusion

Kesimpulan yang didapat adalah menerima H0 yang artinya tidak ada perbedaan yang signifikan antara tingkat kesuksesan iklan pada kelompok control maupun exposed. Maka dari itu perusahaan tidak perlu untuk menggunakan iklan interaktif online karena tidak akan mendapatkan perbedaan yang signifikan jika menggunakan iklan biasa juga. Ditambah lagi iklan interaktif memiliki beberapa kekurangan seperti :
* Biaya yang tergolong mahal karena sulitnya membuat iklan jenis ini dengan mandiri dan memungkinkan perusahaan untuk menyewa spesialis iklan
* Persiapan waktu yang lama karena kesuksesan iklan interaktif bergantung pada persiapan sebelum peluncuran iklan seperti menganalisis kompetitor, mengumpulkan data supaya target audiens bisa tepat
